# 1-CACHE AND PERSIST
# 2-spark-submit

In [1]:
import findspark
findspark.init()

In [2]:
from pyspark.sql import SparkSession

In [3]:

from pyspark.sql.types import *

In [4]:
from pyspark import StorageLevel

In [5]:
spark=SparkSession.builder\
                  .master("local")\
                  .appName('new')\
                  .getOrCreate()
sc=spark.sparkContext

22/11/07 14:37:07 WARN Utils: Your hostname, ashwins-MacBook-Air.local resolves to a loopback address: 127.0.0.1; using 192.168.1.15 instead (on interface en0)
22/11/07 14:37:07 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/11/07 14:37:07 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [5]:

base_rdd = sc.textFile("customerorders-201008-180523.csv")
mapped_input = base_rdd.map(lambda x: (x.split(",")[0],float(x.split(",")[2])))
total_by_customer = mapped_input.reduceByKey(lambda x,y: x+y)
premium_customers = total_by_customer.filter(lambda x: x[1] > 5000)
doubled_amount = premium_customers.map(lambda x:(x[0],x[1]*2)).persist(StorageLevel.MEMORY_ONLY)
result = doubled_amount.collect()
for x in result:
    print(x)
print(doubled_amount.count())


('35', 10310.839999999998)
('2', 11989.18)
('29', 10065.059999999998)
('70', 10736.499999999998)
('85', 11006.86)
('42', 11393.680000000006)
('15', 10827.020000000002)
('22', 10038.899999999998)
('54', 12130.779999999999)
('0', 11049.899999999996)
('40', 10372.859999999999)
('55', 10596.180000000004)
('61', 10994.959999999995)
('6', 10795.759999999997)
('26', 10500.8)
('25', 10115.220000000001)
('71', 11991.320000000005)
('39', 12386.219999999998)
('60', 10081.419999999998)
('97', 11954.37999999999)
('69', 10246.020000000002)
('64', 10577.379999999992)
('72', 10674.88)
('34', 10661.6)
('73', 12412.399999999998)
('8', 11034.480000000001)
('46', 11926.219999999998)
('19', 10118.859999999997)
('65', 10280.699999999997)
('9', 10645.299999999997)
('59', 11285.78)
('90', 10580.819999999996)
('68', 12750.899999999994)
('11', 10304.580000000004)
('41', 11275.24)
('58', 10875.460000000001)
('87', 10412.8)
('17', 10065.359999999999)
('33', 10509.319999999996)
('62', 10506.640000000003)
('92', 10

In [6]:
import os
os.path.abspath("")

'/Users/ashwinpandey/PySparkPractice'

# Example

In [1]:
#find the top rated movies
#1. atleast 1000 people have rated
#2. rating should be > 4.5


In [10]:
ratings_rdd = sc.textFile("ratings-201014-183159.dat")
mapped_rdd = ratings_rdd.map(lambda x: (x.split("::")[1], x.split("::")[2]))
new_mapped_rdd = mapped_rdd.mapValues(lambda x: (float(x),1.0))
reduce_rdd = new_mapped_rdd.reduceByKey(lambda x,y: (x[0]+y[0], x[1]+y[1]))
filtered_rdd = reduce_rdd.filter(lambda x: x[1][0] > 1000)
final_rdd = filtered_rdd.mapValues(lambda x: x[0]/x[1]).filter(lambda x: x[1] > 4.5)


In [13]:
movies_rdd= sc.textFile("movies-201014-183159.dat")
movies_mapped_rdd = movies_rdd.map(lambda x: (x.split("::")[0],(x.split("::")[1],x.split("::")[2])))


In [14]:
joined_rdd = movies_mapped_rdd.join(final_rdd)
top_movies_rdd = joined_rdd.map(lambda x: x[1][0])
result = top_movies_rdd.collect()
for x in result:
    print(x)

('Usual Suspects, The (1995)', 'Crime|Thriller')
('Close Shave, A (1995)', 'Animation|Comedy|Thriller')
('Seven Samurai (The Magnificent Seven) (Shichinin no samurai) (1954)', 'Action|Drama')
('Shawshank Redemption, The (1994)', 'Drama')
("Schindler's List (1993)", 'Drama|War')
('Godfather, The (1972)', 'Action|Crime|Drama')
('Wrong Trousers, The (1993)', 'Animation|Comedy')


# Structured API(dataframe,datasets,SparkSQL)

In [15]:
# DATASETS are not supported in pyspark

In [16]:
#Spark Session is a umbrella which holds all the the library of spark like sparkContext

In [21]:
d1=spark.read.csv("orders-201019-002101.csv")

In [22]:
d1.show()

+--------+--------------------+-----------------+---------------+
|     _c0|                 _c1|              _c2|            _c3|
+--------+--------------------+-----------------+---------------+
|order_id|          order_date|order_customer_id|   order_status|
|       1|2013-07-25 00:00:...|            11599|         CLOSED|
|       2|2013-07-25 00:00:...|              256|PENDING_PAYMENT|
|       3|2013-07-25 00:00:...|            12111|       COMPLETE|
|       4|2013-07-25 00:00:...|             8827|         CLOSED|
|       5|2013-07-25 00:00:...|            11318|       COMPLETE|
|       6|2013-07-25 00:00:...|             7130|       COMPLETE|
|       7|2013-07-25 00:00:...|             4530|       COMPLETE|
|       8|2013-07-25 00:00:...|             2911|     PROCESSING|
|       9|2013-07-25 00:00:...|             5657|PENDING_PAYMENT|
|      10|2013-07-25 00:00:...|             5648|PENDING_PAYMENT|
|      11|2013-07-25 00:00:...|              918| PAYMENT_REVIEW|
|      12|

In [23]:
d1.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- _c1: string (nullable = true)
 |-- _c2: string (nullable = true)
 |-- _c3: string (nullable = true)



In [24]:
d1=spark.read.option("header",True).csv("orders-201019-002101.csv")

In [25]:
d1.show()

+--------+--------------------+-----------------+---------------+
|order_id|          order_date|order_customer_id|   order_status|
+--------+--------------------+-----------------+---------------+
|       1|2013-07-25 00:00:...|            11599|         CLOSED|
|       2|2013-07-25 00:00:...|              256|PENDING_PAYMENT|
|       3|2013-07-25 00:00:...|            12111|       COMPLETE|
|       4|2013-07-25 00:00:...|             8827|         CLOSED|
|       5|2013-07-25 00:00:...|            11318|       COMPLETE|
|       6|2013-07-25 00:00:...|             7130|       COMPLETE|
|       7|2013-07-25 00:00:...|             4530|       COMPLETE|
|       8|2013-07-25 00:00:...|             2911|     PROCESSING|
|       9|2013-07-25 00:00:...|             5657|PENDING_PAYMENT|
|      10|2013-07-25 00:00:...|             5648|PENDING_PAYMENT|
|      11|2013-07-25 00:00:...|              918| PAYMENT_REVIEW|
|      12|2013-07-25 00:00:...|             1837|         CLOSED|
|      13|

In [26]:
#find the total orders placed by each customer where customer id > 10000

In [6]:
orderDf =spark.read.option("header",True).option("inferSchema",True).csv("orders-201019-002101.csv")
orderDf.show(5)
orderDf.printSchema()

+--------+-------------------+-----------+---------------+
|order_id|         order_date|customer_id|   order_status|
+--------+-------------------+-----------+---------------+
|       1|2013-07-25 00:00:00|      11599|         CLOSED|
|       2|2013-07-25 00:00:00|        256|PENDING_PAYMENT|
|       3|2013-07-25 00:00:00|      12111|       COMPLETE|
|       4|2013-07-25 00:00:00|       8827|         CLOSED|
|       5|2013-07-25 00:00:00|      11318|       COMPLETE|
+--------+-------------------+-----------+---------------+
only showing top 5 rows

root
 |-- order_id: integer (nullable = true)
 |-- order_date: timestamp (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- order_status: string (nullable = true)



In [12]:
k=orderDf.collect()
len(k)

68884

In [13]:
for i in k:
    print(i)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



Row(order_id=57495, order_date=datetime.datetime(2014, 7, 23, 0, 0), customer_id=6677, order_status='COMPLETE')
Row(order_id=57496, order_date=datetime.datetime(2014, 7, 23, 0, 0), customer_id=7340, order_status='COMPLETE')
Row(order_id=57497, order_date=datetime.datetime(2014, 7, 23, 0, 0), customer_id=6983, order_status='PENDING_PAYMENT')
Row(order_id=57498, order_date=datetime.datetime(2014, 7, 23, 0, 0), customer_id=3198, order_status='ON_HOLD')
Row(order_id=57499, order_date=datetime.datetime(2014, 7, 23, 0, 0), customer_id=4176, order_status='PENDING_PAYMENT')
Row(order_id=57500, order_date=datetime.datetime(2014, 7, 23, 0, 0), customer_id=5663, order_status='COMPLETE')
Row(order_id=57501, order_date=datetime.datetime(2014, 7, 23, 0, 0), customer_id=4364, order_status='CLOSED')
Row(order_id=57502, order_date=datetime.datetime(2014, 7, 23, 0, 0), customer_id=2441, order_status='COMPLETE')
Row(order_id=57503, order_date=datetime.datetime(2014, 7, 23, 0, 0), customer_id=3559, order_

In [30]:
#inferSchema does not always gives correct value so better to avoid it

In [38]:
Df = orderDf.repartition(4) \
           .where("order_customer_id > 10000") \
           .select("order_id","order_customer_id").groupBy("order_customer_id").count()


In [42]:
Df.show()


+-----------------+-----+
|order_customer_id|count|
+-----------------+-----+
|            10362|    6|
|            12027|    3|
|            11858|    7|
|            10623|    8|
|            11458|    7|
|            11748|    6|
|            12046|    4|
|            11317|    7|
|            11141|    7|
|            10206|    4|
|            10817|    6|
|            10914|    3|
|            10703|    6|
|            10462|    7|
|            10121|    5|
|            12006|    4|
|            10081|    7|
|            11280|    4|
|            11500|    6|
|            10230|    3|
+-----------------+-----+
only showing top 20 rows



In [44]:
print(Df)

DataFrame[order_customer_id: int, count: bigint]


In [45]:
#1. wrong column nameso when we give a column name 
#which does not exist then the error is shown at runtime and not at compile time.


In [48]:
#2.standard way

orderDf = spark.read.format("csv")\
         .option("header",True)\
         .option("inferSchema",True)\
         .option("path","orders-201019-002101.csv")\
         .load()

orderDf.show()

+--------+-------------------+-----------------+---------------+
|order_id|         order_date|order_customer_id|   order_status|
+--------+-------------------+-----------------+---------------+
|       1|2013-07-25 00:00:00|            11599|         CLOSED|
|       2|2013-07-25 00:00:00|              256|PENDING_PAYMENT|
|       3|2013-07-25 00:00:00|            12111|       COMPLETE|
|       4|2013-07-25 00:00:00|             8827|         CLOSED|
|       5|2013-07-25 00:00:00|            11318|       COMPLETE|
|       6|2013-07-25 00:00:00|             7130|       COMPLETE|
|       7|2013-07-25 00:00:00|             4530|       COMPLETE|
|       8|2013-07-25 00:00:00|             2911|     PROCESSING|
|       9|2013-07-25 00:00:00|             5657|PENDING_PAYMENT|
|      10|2013-07-25 00:00:00|             5648|PENDING_PAYMENT|
|      11|2013-07-25 00:00:00|              918| PAYMENT_REVIEW|
|      12|2013-07-25 00:00:00|             1837|         CLOSED|
|      13|2013-07-25 00:0

In [51]:
#NOTE- In case of json file we do not inferschema and header

#orderDf1 = spark.read.format("json")\
#           .option("path","orders-201019-002101.csv")\
 #          .load()


In [52]:
#read modes===========
#1- PERMISSIVE 
#2- DROPMALFORMED 
#3- FAILFAST 

In [ ]:
#SCHEMA ========
# 1. INFER (INFER SCHEMA OPTION)
# 2. IMPLICIT (PARQUET, AVRO ETC..)
#3. EXPLICITStructured 

In [ ]:
# 3 options to have the schema for a dataframe.
# 1. infer schema (inferSchema as true)
# 2. implicit schema (reading parquet, avro etc...) 
# 3. explicit schema (manually defining the schema) 
#explicit schema ================= 
#1. Programatically using StructType
#2. DDL String  Programatic approach ====================== 

# Programatically using StructType and DDL String

In [69]:
ordersSchema = StructType([ StructField("orderid", IntegerType()),
                               StructField("orderdate", TimestampType()),
                               StructField("customerid", IntegerType()),
                               StructField("status", StringType())])


In [68]:
ordersSchemaDDL = "orderid Int, orderdate String, custid Int, ordstatus String"